### 1. General info of dataset GSE141460

This is the Jupyter Notebook for dataset GSE141460. Its dataset includes barcodes/genes/matrix files for each sample from 10X, as well as a txt file for samples from Smart-seq2. There are 5/6 paediatric samples from 10X, and 23 paediatric samples from Smart-seq2.

In [12]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes

1. `barcodes.tsv`: cell barcodes, which go into `.obs`
2. `genes.tsv`: gene names, `.var`
3. `matrix.mtx`: the expression matrix, `.X`

In [13]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE141460')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')

tissue_subgroup_age_sex_progression_dict = {'MUV014': ['medulla_oblongata', 'PFA-1a', 1.9, 'male', 'recurrence'],
                                            'MUV013': ['posterior_fossa', 'PFA-2a', 2.2, 'female', 'diagnosis'],
                                            'MUV051': ['medulla_oblongata', 'PFA-2b', 2.8, 'male', 'recurrence'],
                                            'MUV056': ['temporal_right', 'ST-RELA', 8.6, 'male', 'recurrence'],
                                            'MUV006': ['parietal_left', 'ST-YAP1', 0.3, 'female', 'diagnosis']}

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_name_muv = sample_name.split('_')[1]
    sample_h5ad = sample_name + '_uni.h5ad'

    if sample_name_muv in tissue_subgroup_age_sex_progression_dict:
        sample = sc.read_10x_mtx(
            sample_directory,
            var_names='gene_symbols',  
            cache=False
        )

        # Create an observation metric info to store related features
        obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
        obs_metrics['cancer_type'] = tissue_subgroup_age_sex_progression_dict[sample_name_muv][1] + '_ependymoma'
        obs_metrics['dataset'] = 'GSE141460'
        obs_metrics['tissue'] = tissue_subgroup_age_sex_progression_dict[sample_name_muv][0]
        obs_metrics['age'] = tissue_subgroup_age_sex_progression_dict[sample_name_muv][2]
        obs_metrics['sex'] = tissue_subgroup_age_sex_progression_dict[sample_name_muv][3]
        obs_metrics['disease_progression'] = tissue_subgroup_age_sex_progression_dict[sample_name_muv][4]
        obs_metrics['sample_barcode'] = sample_name
        obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
        
        sample.obs = obs_metrics
        sample.obs.set_index("uni_barcode", drop=False, inplace=True)
        print(sample)

        # save the anndata object
        output_path = write_directory.joinpath(sample_h5ad)
        sample.write_h5ad(output_path, compression="gzip")

In [14]:
sample.var

AttributeError: 'str' object has no attribute 'var'

In [ ]:
sample.obs

,cancer_type,dataset,tissue,age,sex,disease_progression,sample_barcode,uni_barcode
uni_barcode,,,,,,,,
GSE141460_AAACCCAAGCGACTAG-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_AAACCCAAGCGACTAG-1
GSE141460_AAACCCAAGGAGCTGT-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_AAACCCAAGGAGCTGT-1
GSE141460_AAACCCAAGTGGCGAT-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_AAACCCAAGTGGCGAT-1
GSE141460_AAACCCACACGTAGAG-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_AAACCCACACGTAGAG-1
GSE141460_AAACCCACAGGTGTGA-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_AAACCCACAGGTGTGA-1
...,...,...,...,...,...,...,...,...
GSE141460_TTTGTTGCATTCATCT-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_TTTGTTGCATTCATCT-1
GSE141460_TTTGTTGGTAGGCTGA-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_TTTGTTGGTAGGCTGA-1
GSE141460_TTTGTTGGTCCTACAA-1,PFA-1a_ependymoma,GSE141460,medulla_oblongata,1.9,male,recurrence,GSM5273079_MUV014,GSE141460_TTTGTTGGTCCTACAA-1


<span style="color:red">**IMPORTANT:**</span> transpose the DataFrame.values to match the AnnData.X

1. `DataFrame.columns`: cell barcodes, which go into `.obs`
2. `DataFrame.index`: gene names, `.var`
3. `DataFrame.values`: the transpose of the expression matrix, `.X`

In [39]:
# Inspect the txt files to find out the file containing paediatric samples
from pathlib import Path

data_dir = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE141460/GSE141460_EPN_raw_counts_meta')

counts_metadata = {
    'Matched': ['Matched_pair_counts_200519lj.txt', 'Matched_pair_metadata_200519lj.txt'],
    'PF_EPN': ['PF_EPN_counts_200519lj.txt', 'PF_EPN_metadata_200519lj.txt'],
    'SP_EPN': ['SP_EPN_counts_200519lj.txt', 'SP_EPN_metadata_200519lj.txt'],
    'ST_EPN': ['ST_EPN_counts_200519lj.txt', 'ST_EPN_metadata_200519lj.txt'],
}

counts = {}
metadatas = {}
for sample, sample_files in counts_metadata.items():
    count = pd.read_csv(data_dir / sample_files[0], sep='\t', index_col=0)
    metadata = pd.read_csv(data_dir / sample_files[1], sep='\t', index_col=0)
    
    print(sample, 'has samples: ', metadata['sample'].unique())
    
    # Save the data into dicts
    counts[sample] = count
    metadatas[sample] = metadata

    

Matched has samples:  ['WEPN1DiaANGANuc' 'WEPN1RecDUDRNuc' 'WEPN20DiaVEMI00Nuc'
 'WEPN20RecVEMI01Nuc']
PF_EPN has samples:  ['BT1313' 'BT1334' 'BT1412Nuc' 'BT1480Nuc' 'MUV013' 'MUV014' 'MUV018'
 'MUV021' 'MUV038' 'MUV051' 'MUV052' 'MUV053' 'MUV063' 'MUV071']
SP_EPN has samples:  ['BT1678' 'MUV068']
ST_EPN has samples:  ['BT1030' 'CPDM0785' 'MUV006' 'MUV043' 'MUV043Nuc1' 'MUV043Nuc2' 'MUV056'
 'Peds4']


In [40]:
counts['Matched']

,WEPN1DiaANGANuc.P1.A01,WEPN1DiaANGANuc.P1.A02,WEPN1DiaANGANuc.P1.A04,WEPN1DiaANGANuc.P1.A05,WEPN1DiaANGANuc.P1.A06,WEPN1DiaANGANuc.P1.A07,WEPN1DiaANGANuc.P1.A08,WEPN1DiaANGANuc.P1.A09,WEPN1DiaANGANuc.P1.A10,WEPN1DiaANGANuc.P1.A11,...,WEPN20RecVEMI01Nuc.P1.H03,WEPN20RecVEMI01Nuc.P1.H04,WEPN20RecVEMI01Nuc.P1.H05,WEPN20RecVEMI01Nuc.P1.H06,WEPN20RecVEMI01Nuc.P1.H07,WEPN20RecVEMI01Nuc.P1.H08,WEPN20RecVEMI01Nuc.P1.H09,WEPN20RecVEMI01Nuc.P1.H10,WEPN20RecVEMI01Nuc.P1.H11,WEPN20RecVEMI01Nuc.P1.H12
A1BG,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00
A1BG-AS1,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,1.00,0.00,0.00
A1CF,0.00,0.0,0.00,0.00,0.00,1.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00
A2M,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,6.99,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00
A2M-AS1,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,5.86,0.0,2.01,1.00,1.21,2.0,2.00,0.00,0.0,0.00,...,1.0,2.0,1.03,1.0,0.0,1.00,0.0,4.89,1.05,3.03
ZYG11B,4.31,6.3,1.00,0.96,0.00,0.0,3.01,1.01,0.0,2.05,...,1.0,0.0,1.04,0.0,0.0,2.23,0.0,6.00,0.00,0.00
ZYX,0.00,0.0,0.00,0.00,0.00,0.0,0.00,22.00,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00
ZZEF1,0.00,0.0,0.00,0.00,78.00,0.0,0.00,0.00,40.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00


In [41]:
metadatas['Matched']

,sample,subtype,malignant,annotation
WEPN1DiaANGANuc.P1.A01,WEPN1DiaANGANuc,PF-A,Malignant,PF-Astroependymal-like
WEPN1DiaANGANuc.P1.A02,WEPN1DiaANGANuc,PF-A,Malignant,PF-Ependymal-like
WEPN1DiaANGANuc.P1.A04,WEPN1DiaANGANuc,PF-A,Malignant,PF-Immune-Reactive
WEPN1DiaANGANuc.P1.A05,WEPN1DiaANGANuc,PF-A,Malignant,PF-Astroependymal-like
WEPN1DiaANGANuc.P1.A06,WEPN1DiaANGANuc,PF-A,Normal,Oligodendrocyte
...,...,...,...,...
WEPN20RecVEMI01Nuc.P1.H08,WEPN20RecVEMI01Nuc,PF-A,Malignant,PF-Glial-Progenitor-like
WEPN20RecVEMI01Nuc.P1.H09,WEPN20RecVEMI01Nuc,PF-A,Malignant,PF-G2M-Phase
WEPN20RecVEMI01Nuc.P1.H10,WEPN20RecVEMI01Nuc,PF-A,Malignant,PF-G2M-Phase
WEPN20RecVEMI01Nuc.P1.H11,WEPN20RecVEMI01Nuc,PF-A,Malignant,PF-Astroependymal-like


According to Table S1 from the paper:

Invalid sample: 

`PF_EPN`: `MUV052`, `MUV063`, 

`ST_EPN`: `MUV043`, `MUV043Nuc1`, `MUV043Nuc2`, 

In [42]:
# Subset the count matrix to exclude these invalid samples
print('PF_EPN has cells: ', counts['PF_EPN'].shape[1])
counts['PF_EPN'] = counts['PF_EPN'].loc[:, ~counts['PF_EPN'].columns.str.startswith('MUV052')]
print('After filtering out MUV052 cells, PF_EPN has cells: ', counts['PF_EPN'].shape[1])
counts['PF_EPN'] = counts['PF_EPN'].loc[:, ~counts['PF_EPN'].columns.str.startswith('MUV063')]
print('After filtering out MUV063 cells, PF_EPN has cells: ', counts['PF_EPN'].shape[1])

PF_EPN has cells:  3983
After filtering out MUV052 cells, PF_EPN has cells:  3681
After filtering out MUV063 cells, PF_EPN has cells:  3423


In [43]:
print('ST_EPN has cells: ', counts['ST_EPN'].shape[1])
counts['ST_EPN'] = counts['ST_EPN'].loc[:, ~counts['ST_EPN'].columns.str.startswith('MUV043')]
print('After filtering out MUV043 cells, ST_EPN has cells: ', counts['ST_EPN'].shape[1])

ST_EPN has cells:  2021
After filtering out MUV043 cells, ST_EPN has cells:  1056


In [44]:
# Create the AnnData objects
write_dir = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')
for sample in counts_metadata:
    print(sample)
    input = counts[sample]

    matrix = scipy.sparse.csr_matrix(input.values.T)
    obs_name = pd.DataFrame(index=input.columns)
    var_name = pd.DataFrame(index=input.index)
    var_name.rename_axis('gene_symbols', inplace=True)

    adata = anndata.AnnData(X=matrix, obs=obs_name, var=var_name)

    # Create an observation metric info to store related features
    obs_metrics = metadatas[sample].copy()
    obs_metrics['cancer_type'] = 'ependymoma'
    obs_metrics['dataset'] = 'GSE141460'
    obs_metrics['tissue'] = 'brain'
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    obs_metrics.rename(columns={'subtype': 'cancer_subtype', 'sample': 'sample_barcode', 
                                'malignant': 'malignant_from_paper', 'annotation': 'cell_type_from_paper'}, inplace=True)

    adata.obs = adata.obs.merge(obs_metrics, how='inner', left_index=True, right_index=True)
    adata.obs.set_index("uni_barcode", drop=False, inplace=True)
    print(adata)

    # save the anndata object
    output_file = sample + '_uni.h5ad'
    output_path = write_dir / output_file
    adata.write_h5ad(output_path, compression="gzip")

Matched
AnnData object with n_obs × n_vars = 652 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
PF_EPN
AnnData object with n_obs × n_vars = 3423 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
SP_EPN
AnnData object with n_obs × n_vars = 338 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
ST_EPN
AnnData object with n_obs × n_vars = 1056 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'


In [45]:
adata.obs

,sample_barcode,cancer_subtype,malignant_from_paper,cell_type_from_paper,cancer_type,dataset,tissue,uni_barcode
uni_barcode,,,,,,,,
GSE141460_BT1030.P1.A01,BT1030,ST-Midline,Malignant,ST-Midline,ependymoma,GSE141460,brain,GSE141460_BT1030.P1.A01
GSE141460_BT1030.P1.A03,BT1030,ST-Midline,Malignant,ST-Midline,ependymoma,GSE141460,brain,GSE141460_BT1030.P1.A03
GSE141460_BT1030.P1.A04,BT1030,ST-Midline,Malignant,ST-Ependymal-like,ependymoma,GSE141460,brain,GSE141460_BT1030.P1.A04
GSE141460_BT1030.P1.A06,BT1030,ST-Midline,Malignant,ST-Ependymal-like,ependymoma,GSE141460,brain,GSE141460_BT1030.P1.A06
GSE141460_BT1030.P1.A07,BT1030,ST-Midline,Normal,Microglia,ependymoma,GSE141460,brain,GSE141460_BT1030.P1.A07
...,...,...,...,...,...,...,...,...
GSE141460_Peds4.P6.G06,Peds4,ST-RELA,Malignant,ST-Metabolic,ependymoma,GSE141460,brain,GSE141460_Peds4.P6.G06
GSE141460_Peds4.P6.H02,Peds4,ST-RELA,Normal,OPC,ependymoma,GSE141460,brain,GSE141460_Peds4.P6.H02
GSE141460_Peds4.P6.H05,Peds4,ST-RELA,Malignant,ST-Interferon-Response,ependymoma,GSE141460,brain,GSE141460_Peds4.P6.H05


In [46]:
adata.var

""
gene_symbols
A1BG
A1BG-AS1
A1CF
A2M
A2M-AS1
...
ZYG11A
ZYG11B
ZYX


### 3. Confirmation of created AnnData objects

In [9]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample = anndata.read_h5ad(file)
    print(sample)

AnnData object with n_obs × n_vars = 15906 × 58347
    obs: 'cancer_type', 'dataset', 'tissue', 'age', 'sex', 'disease_progression', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'


AnnData object with n_obs × n_vars = 21968 × 58347
    obs: 'cancer_type', 'dataset', 'tissue', 'age', 'sex', 'disease_progression', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 3193 × 58347
    obs: 'cancer_type', 'dataset', 'tissue', 'age', 'sex', 'disease_progression', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 4015 × 58347
    obs: 'cancer_type', 'dataset', 'tissue', 'age', 'sex', 'disease_progression', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 10529 × 58347
    obs: 'cancer_type', 'dataset', 'tissue', 'age', 'sex', 'disease_progression', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'


In [48]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')

# Loop through all files in the directory
for file in write_directory.iterdir():
    if 'GSM' not in file.name and '_uni.h5ad' in file.name:
        sample = anndata.read_h5ad(file)
        print(sample)

AnnData object with n_obs × n_vars = 3423 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
AnnData object with n_obs × n_vars = 338 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
AnnData object with n_obs × n_vars = 1056 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
AnnData object with n_obs × n_vars = 652 × 23686
    obs: 'sample_barcode', 'cancer_subtype', 'malignant_from_paper', 'cell_type_from_paper', 'cancer_type', 'dataset', 'tissue', 'uni_barcode'


### 4. Convert AnnData objects to SingleCellExperiment objects

In [12]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        # print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

In [13]:
print(sample_sce)

class: SingleCellExperiment 
dim: 58347 10529 
metadata(0):
assays(1): X
rownames(58347): DDX11L1 WASH7P ... U6-36 U1-15
rowData names(1): gene_ids
colnames(10529): GSE141460_AAACCCAAGCGACTAG-1
  GSE141460_AAACCCAAGGAGCTGT-1 ... GSE141460_TTTGTTGTCGCGCTGA-1
  GSE141460_TTTGTTGTCTGTCAGA-1
colData names(8): cancer_type dataset ... sample_barcode uni_barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):



In [49]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE141460')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if 'GSM' not in file.name and '_uni.h5ad' in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        # print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

In [50]:
print(sample_sce)

class: SingleCellExperiment 
dim: 23686 652 
metadata(0):
assays(1): X
rownames(23686): A1BG A1BG-AS1 ... ZZEF1 ZZZ3
rowData names(0):
colnames(652): GSE141460_WEPN1DiaANGANuc.P1.A01
  GSE141460_WEPN1DiaANGANuc.P1.A02 ...
  GSE141460_WEPN20RecVEMI01Nuc.P1.H11
  GSE141460_WEPN20RecVEMI01Nuc.P1.H12
colData names(8): sample_barcode cancer_subtype ... tissue uni_barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

